In [203]:
import pandas as pd
import matplotlib.pyplot as plt
import collections
import numpy as np
from scipy import stats
from difflib import SequenceMatcher

In [204]:
# load xlsx data
data = pd.read_excel('./mpg.data.xlsx')

In [205]:
# used to select correct columns
data = data[['mpg', 'cylinders', 'displayments', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'car name']]

In [206]:
# used to extract car brands from car name
def getBrand(x):
    brand = x.lower().split()[0]
    return brand
data['brand'] = data['car name'].map(getBrand)

In [207]:
# used to create bag of words from car brands in order to find typos
bagOfWords = {}
for name in data['brand']:
    words = name.lower().replace('(', '').replace(')', '').split()
    for word in words:
        if word not in bagOfWords:
            bagOfWords[word] = 0
        bagOfWords[word] += 1

In [208]:
# used to find typos
matchStrictness = 0.7
foundWords = []
for word in bagOfWords:
    for altWord in bagOfWords:
        if word in foundWords or altWord in foundWords:
            continue
        if word.isdigit() or altWord.isdigit():
            continue
        if word == altWord:
            continue
        similarity = SequenceMatcher(None, word, altWord)
        if similarity.ratio() > matchStrictness:
            foundWords.append(word)
            foundWords.append(altWord)
            print(f'{word} - {altWord}')

chevrolet - chevroelt
toyota - toyouta
volkswagen - vokswagen
mazda - maxda
mercedes-benz - mercedes


In [209]:
# used to fix typos in brand name column
def getBrand(x):
    brand = x.lower().split()[0]
    if (brand == 'vw' or brand == 'vokswagen'):
        return 'volkswagen'
    if (brand == 'toyouta'):
        return 'toyota'
    if (brand == 'maxda'):
        return 'mazda'
    if (brand == 'chevroelt'):
        return 'chevrolet'
    if (brand == 'mercedes-benz'):
        return 'mercedes'
    return brand
data['brand'] = data['car name'].map(getBrand)

In [210]:
# used to create new columns based on the existance of the following tokens in the 'car name'
carTokens = ['sw', 'custom', 'brougham', 'diesel', 'turbo', 'classic', 'dl', 'coupe', 'deluxe', 'sport', 'pickup', 'auto', 'man', '2+2']

for token in carTokens:
    def tokenExistsInName(x):
        name = x.lower()
        return token in name
    data[token] = data['car name'].map(tokenExistsInName)

In [211]:
data

,mpg,cylinders,displayments,horsepower,weight,acceleration,model year,origin,car name,brand,...,turbo,classic,dl,coupe,deluxe,sport,pickup,auto,man,2+2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet,...,False,False,False,False,False,False,False,False,False,False
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,buick,...,False,False,False,False,False,False,False,False,False,False
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,plymouth,...,False,False,False,False,False,False,False,False,False,False
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,amc,...,False,False,False,False,False,False,False,False,False,False
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,ford,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl,ford,...,False,False,False,False,False,False,False,False,False,False
402,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup,volkswagen,...,False,False,False,False,False,False,True,False,False,False
403,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage,dodge,...,False,False,False,False,False,False,False,False,False,False
404,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger,ford,...,False,False,False,False,False,False,False,False,False,False


In [212]:
# Use SVR regressor to fix missing values in horsepower

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

df = data.dropna()

sourceData = df[['mpg', 'acceleration','cylinders', 'displayments', 'weight']]
targetData = df[['horsepower']]

xTrain, xTest, yTrain, yTest = train_test_split(sourceData, targetData)
horsePowerClassifier = SVR(kernel='linear')
horsePowerClassifier = horsePowerClassifier.fit(xTrain, yTrain.values.flatten())
score = horsePowerClassifier.score(xTest, yTest.values.flatten())
print(f'score: {score}')

score: 0.8802611840925155


In [213]:
# used to fill missing horsepower values
for rowId in data.index[data['horsepower'].isna()]:
    row = data.iloc[rowId]
    values = row[['mpg', 'acceleration','cylinders', 'displayments', 'weight']]
    prediction = horsePowerClassifier.predict([values])[0]
    data.at[rowId, 'horsepower'] = prediction

data.isna().sum()

mpg             8
cylinders       0
displayments    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
brand           0
sw              0
custom          0
brougham        0
diesel          0
turbo           0
classic         0
dl              0
coupe           0
deluxe          0
sport           0
pickup          0
auto            0
man             0
2+2             0
dtype: int64

In [214]:
data.to_csv("extracted.data.csv")

### 